In [1]:
import sys
sys.path.append("/home/ly/workspace/mmsa")
seed = 1000003
import numpy as np
import torch
from torch import nn
from torch import optim

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
from models.text_cnn import *
from utils.train import *
from typing import *
from collections import Counter

from utils.load_raw_yelp import *
from utils.dataset import *
from utils.train import *
config

{'use_cuda': True,
 'task': 'CLS',
 'embedding_dim': 100,
 'embedding': None,
 'freeze_embedding': True,
 'filter_size': (2, 3, 4, 5),
 'filter_num': 50,
 'max_tokens': 512,
 'bias_init': 1.0,
 'use_img': True,
 'img_input_size': 4096,
 'img_output_size': 100,
 'img_num': 3,
 'output_size': 5,
 'dropout': 0.5}

In [2]:
config["use_img"] = False
config

{'use_cuda': True,
 'task': 'CLS',
 'embedding_dim': 100,
 'embedding': None,
 'freeze_embedding': True,
 'filter_size': (2, 3, 4, 5),
 'filter_num': 50,
 'max_tokens': 512,
 'bias_init': 1.0,
 'use_img': False,
 'img_input_size': 4096,
 'img_output_size': 100,
 'img_num': 3,
 'output_size': 5,
 'dropout': 0.5}

In [3]:
%%time
batch_size = 64
workers = 4
train_set, valid_set, test_set= load_glove_data(config)
train_loader, valid_loader, test_loader = get_yelp_raw_loader(batch_size, workers, get_collate_fn(config), train_set, valid_set, test_set)
model = Model(config).cuda()
loss = nn.CrossEntropyLoss()
print(get_parameter_number(model))

_interval = 5
lr = 1e-3
epoches = 50
stoping_step = 5
optimizer = get_regal_optimizer(model, optim.AdamW, lr)

viz = get_Visdom()
batch_loss_drawer = VisdomScalar(viz, f"batch_loss interval:{_interval}")
epoch_loss_drawer = VisdomScalar(viz, f"Train and valid loss", 2)
acc_drawer = VisdomScalar(viz, "Train and valid accuracy", 2)
text_writer = VisdomTextWriter(viz, "Training")

batch_loss = []
train_loss = []
valid_loss = []
train_acc = []
valid_acc = []

res, model = train_visdom_v2(model, optimizer, loss, viz, train_loader,
                          valid_loader, epoches, batch_loss, batch_loss_drawer,
                          train_loss, valid_loss, epoch_loss_drawer,
                          train_acc, valid_acc, acc_drawer, text_writer,
                         _interval=_interval, early_stop=stoping_step)

eval_model_on_raw_yelp(model, loss, test_loader)

Setting up a new session...


{'Total': 4289505, 'Trainable': 71205}


No 1 epoch: 554it [00:16, 32.68it/s]
No 2 epoch: 554it [00:16, 32.64it/s]
No 3 epoch: 554it [00:16, 33.01it/s]
No 4 epoch: 554it [00:17, 32.12it/s]
No 5 epoch: 554it [00:16, 32.70it/s]
No 6 epoch: 554it [00:17, 32.50it/s]
No 7 epoch: 554it [00:17, 32.00it/s]
No 8 epoch: 554it [00:17, 31.92it/s]
No 9 epoch: 554it [00:17, 32.19it/s]
No 10 epoch: 554it [00:17, 31.33it/s]
No 11 epoch: 554it [00:17, 32.12it/s]
No 12 epoch: 554it [00:17, 31.63it/s]
No 13 epoch: 554it [00:17, 31.48it/s]


CPU times: user 2min 25s, sys: 17.9 s, total: 2min 43s
Wall time: 4min 9s


{'Boston': {'Accuracy': 0.5555555555555556,
  'F1': 0.5477660777875244,
  'Loss': 1.0528017373312086},
 'Chicago': {'Accuracy': 0.5538461538461539,
  'F1': 0.5475804982816275,
  'Loss': 1.060893373122582},
 'Los Angeles': {'Accuracy': 0.5439678284182305,
  'F1': 0.5381805476619441,
  'Loss': 1.0767751831791037},
 'New York': {'Accuracy': 0.5422740524781341,
  'F1': 0.5330300329701403,
  'Loss': 1.0581068836912817},
 'San Francisco': {'Accuracy': 0.5298245614035088,
  'F1': 0.5245073629849208,
  'Loss': 1.0703527751721833},
 'Total': {'Accuracy': 0.543350864012021,
  'F1': 0.5367667077167395,
  'Loss': 1.0695039390174184}}